# Situation

At the time of this experiment, Udacity courses currently have two options on the course overview page: "start free trial", and "access course materials". If the student clicks "start free trial", they will be asked to enter their credit card information, and then they will be enrolled in a free trial for the paid version of the course. After 14 days, they will automatically be charged unless they cancel first. If the student clicks "access course materials", they will be able to view the videos and take the quizzes for free, but they will not receive coaching support or a verified certificate, and they will not submit their final project for feedback.

# Experiment

In the experiment, Udacity tested a change where if the student clicked "start free trial", they were asked how much time they had available to devote to the course. If the student indicated 5 or more hours per week, they would be taken through the checkout process as usual. If they indicated fewer than 5 hours per week, a message would appear indicating that Udacity courses usually require a greater time commitment for successful completion, and suggesting that the student might like to access the course materials for free. At this point, the student would have the option to continue enrolling in the free trial, or access the course materials for free instead. [This screenshot](https://www.google.com/url?q=https://drive.google.com/a/knowlabs.com/file/d/0ByAfiG8HpNUMakVrS0s4cGN2TjQ/view?usp%3Dsharing&sa=D&ust=1566807927543000) shows what the experiment looks like.

The unit of diversion is a cookie, although if the student enrolls in the free trial, they are tracked by user-id from that point forward. The same user-id cannot enroll in the free trial twice. For users that do not enroll, their user-id is not tracked in the experiment, even if they were signed in when they visited the course overview page.

## Hypothesis

$H_0$: This change won't set clearer expectations for students upfront, and not reduce the number of frustrated students who leave the free trial because they don't have enough time. It won't significantly reduce the number of students to continue past the free trial and eventually complete the course.

$H_1$: This change will set clearer expectations for students upfront, thus reducing the number of frustrated students who leave the free trial because they don't have enough time—without significantly reducing the number of students to continue past the free trial and eventually complete the course. If this hypothesis holds true, Udacity could improve the overall student experience and improve coaches' capacity to support students who are likely to complete the course.

## Metric Choice

There are several possible metrics that could be used for the experiment.

### Invariant Metrics

Invariant metrics are chosen to investigate possible issues in the experiment setup and execution, i.e. detect bad measurements due to errors or unintended consequences.

#### Selected:    

| Metric Name               | Formula                                 | $d_{min}$ | Notation         |
|---------------------------|-----------------------------------------|-----------|------------------|
| Number of cookies         | # cookies on course overview page       | 3000      | $cookies_{uniq}$ |
| Number of clicks          | # cookies clicked on button             | 240       | $clicks_{uniq}$  |
| Click-Through-Probability | $\frac{clicks_{uniq}}{cookies_{uniq}}$  | 1%        | $CTP$            |

- **Number of cookies**: That is, number of unique cookies to view the course overview page. (dmin=3000)
    - A good invariant as this is the unit of diversion, hence it is randomized by definition. This metric should not be significantly different from a p=0.5 value (for an equal split)
- **Number of clicks**: That is, number of unique cookies to click the "Start free trial" button (which happens before the free trial screener is trigger). (dmin=240)
    - Also a good invariant, even though it is a subset of cookies from above the experiment happens after the button is clicked and therefore the cookies should not be significantly different between groups.
- **Click-through-probability**: That is, number of unique cookies to click the "Start free trial" button divided by number of unique cookies to view the course overview page. (dmin=0.01)
    - The click-through-probability from course overview page to "Start free trial" click should be unaffected by the experiment, hence not significantly different between groups.

#### Not Selected:    

| Metric Name               | Formula                                  | Practical Significance $d_{min}$ | Notation |
|---------------------------|------------------------------------------|----------------------------------|----------|
| Number of user-ids        | # user-ids that enroll in the free trial | 50                               |$enrolled$|

- **Number of user-ids**: That is, number of users who enroll in the free trial. (dmin=50)
    - This could possibly have been an invariant metric but since it is only recorded after the experimental change we're unable to use it as such. It might be affected by the variation.

### Evaluation Metrics

Evaluation metrics are chosen to investigate the impact of the changes. They are usually tied to business goals (at least indirectly).

| Metric Name      | Formula                           | Practical Significance $d_{min}$ | Notation             |
|------------------|-----------------------------------|----------------------------------|----------------------|
| Gross Conversion | $\frac{enrolled}{cookies_{uniq}}$ | 1%                               | $Conversion_{gross}$ |
| Retention        | $\frac{payment}{enrolled}$        | 1%                               | $Retention$      |
| Net Conversion   | $\frac{payment}{cookies_{uniq}}$  | 0.75%                            | $Conversion_{net}$   |


- **Gross conversion**: That is, number of user-ids to complete checkout and enroll in the free trial divided by number of unique cookies to click the "Start free trial" button. (dmin= 0.01)
    - This metric is going to measure whether we're successfully deterring users from enrolling in the free trial. We're expecting the gross conversion to go down ($H_1$).
- **Retention**: That is, number of user-ids to remain enrolled past the 14-day boundary (and thus make at least one payment) divided by number of user-ids to complete checkout. (dmin=0.01)
    - This metric is going to measure the probability that we're successfully retaining users through the free trial. We're expecting retention to increase ($H_1$).
- **Net conversion**: That is, number of user-ids to remain enrolled past the 14-day boundary (and thus make at least one payment) divided by the number of unique cookies to click the "Start free trial" button. (dmin= 0.0075)
    - This metric is going to measure whether we remain able to funnel users through the free trial that are likely to succeed in completing a course. We're hoping that this metric will stay unchanged ($H_1$).

NOTE: _Any place "unique cookies" are mentioned, the uniqueness is determined by day. (That is, the same cookie visiting on different days would be counted twice.) User-ids are automatically unique since the site does not allow the same user-id to enroll twice._

NOTE: $d_{min}$ is the difference necessary to be practically significant.

**Essential Measures for Launch Recommendation:** It is necessary to see a significant drop by at least 1% in Gross Conversion, without significantly (1% and 0.75% respectively) affecting Retention and Net Conversion. 

